# Exercises

In [20]:
import logging

In [100]:
from optibook.synchronous_client import Exchange
import datetime

from optibook.synchronous_client import InfoOnly


import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


# Instrument

In [101]:
instrument_id = 'PHILIPS_A'

In [146]:
e = Exchange()
a = e.connect()

2021-01-23 16:10:41,998 [asyncio   ] [MainThread  ] Using selector: EpollSelector


## Current Possitoin

In [147]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

PHILIPS_A -140
PHILIPS_B 147


In [148]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

PHILIPS_A {'volume': -140, 'cash': 10913.199999999999}
PHILIPS_B {'volume': 147, 'cash': -11441.900000000001}


In [151]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

38.299999999995634


## Trading History

In [27]:
book = e.get_last_price_book(instrument_id)
print(book)

In [30]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

[PHILIPS_A] price(71.0), volume(1), aggressor_side(ask), buyer(), seller()


In [33]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [12]:
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [38]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

### Making a trade

To make it save, sell one at 0.02 

In [63]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=0.02, volume=1, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 194540


In [61]:
orders = e.get_outstanding_orders(instrument_id); orders

{194518: <optibook.common_types.OrderStatus at 0x7fa04c76dbe0>}

In [62]:
for o in orders.keys():
    print('Deleting {}'.format(o))
    e.delete_order(instrument_id, order_id=o)

Deleting 194518


In [71]:
book = i.get_last_price_book(instrument_id)

In [70]:
i = InfoOnly()
#i = InfoOnly(host='host-to-connect-to')
i.connect()

2021-01-23 14:19:48,248 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [15]:
disp_tradeticks = []
disp_tradeticks_max_age = datetime.timedelta(seconds=20)
feedcode = 'PHILIPS_A'

In [88]:
book = i.get_last_price_book(feedcode)
book

In [68]:
e = Exchange()
a = e.connect()
tradeticks = e.poll_new_trade_ticks(instrument_id)

2021-01-23 14:14:10,089 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [87]:
book

```
If there exists ask.val < bid.val:
    x = min( ask.ammount, bid.ammount)
    buy (ask.val, x)
    sell bid(bid.val , x)
```

# Book info

In [95]:
e = Exchange()
a = e.connect()      

2021-01-23 14:39:11,488 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [104]:
i = InfoOnly()
i.connect()
bookA = i.get_last_price_book('PHILIPS_A')  
bookB = i.get_last_price_book('PHILIPS_B')  

2021-01-23 14:42:02,627 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [105]:
book.bids

[[price_volume] price=72.10000000000001, volume=200]

In [106]:
best_askA = bookA.asks[0]
best_bidA = bookA.bids[0]
best_askB = bookB.asks[0]
best_bidB = bookB.bids[0]

In [123]:
best_askA.price = 1
best_bidB.price = 2

In [133]:
if min(best_askA.price, best_askB.price) < max(best_bidA.price, best_bidB.price):
    
    if best_askA.price < best_askB.price:
        s = 0
        vol = np.min([best_askA.volume, best_bidB.volume])
    else:
        s = 1
        vol = np.min([best_askB.volume, best_bidA.volume])
    
    options = ['PHILIPS_A', 'PHILIPS_B']
    
    
    if True:
        print('Buy {}, Sell {}, Volume {}'.format(options[s],options[(s+1)%2], vol) )
        
    else:
        e.insert_order(options[s], price=0.02, volume=vol, side='bid', order_type='limit')
        e.insert_order(options[(s+1)%2], price=0.02, volume=vol, side='ask', order_type='limit')
    
else:
    pass

Buy PHILIPS_A, Sell PHILIPS_B, Volume 60
1 76.10000000000001 75.5 2


In [130]:
print(book_pretty(bookA), book_pretty(bookB), sep='\n')




        |    300.00  |      1 
        |    100.00  |      1 
        |      1.00  |    200 
------------------------------
   200  |     75.50  |       










        |     76.40  |     40 
        |     76.10  |     20 
------------------------------
    60  |      2.00  |       








In [132]:
bookA.asks

[[price_volume] price=1, volume=200,
 [price_volume] price=100.0, volume=1,
 [price_volume] price=300.0, volume=1]

# Pretty book


In [127]:

def book_pretty(price_book):
    if price_book is None:
        return 'NO BOOK'
    
    res = ''
    for level in reversed(range(6)):
        if level >= len(price_book.asks):
            res += "\n"
        else:
            ask = price_book.asks[level]
            res += f"        |  {ask.price:8.2f}  |  {ask.volume:5} \n"

    res += '------------------------------\n'

    for level in range(6):
        if level >= len(price_book.bids):
            res += "\n"
        else:
            bid = price_book.bids[level]
            res += f" {bid.volume:5}  |  {bid.price:8.2f}  |       \n"   

    return res

# Timing

In [140]:
import time
import sleep

ModuleNotFoundError: No module named 'sleep'

In [ ]:
comm = 

In [144]:
start = time.time()        
time.sleep(.001)
end = time.time()

print(f'Took {end-start} seconds')

Took 0.0012171268463134766 seconds
